In [2]:
#importing the required libraries
import numpy as np
import tensorflow as tf

In [3]:
#importing data and storing 
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
#defining weight and bias functions used to assign random weight and fixed bias for the initial forward propogation
def weight(a):
    return tf.Variable(tf.truncated_normal(a, stddev=0.1))

def bias(a):
    return tf.Variable(tf.constant(0.1, shape=a))

In [5]:
#defining the convolution function
def cnn(x):
#conversion of input pixels in a 3d tensor
    input_layer = tf.reshape(x, [-1,28,28,1])
#passing this to the first layer using kernels of size 5*5 and 32 in number
#a rectified linear unit is applied to prevent the derivative from vanishing 
    conv1 = tf.nn.relu(tf.nn.conv2d(input_layer, weight([5,5,1,32]), strides=[1, 1, 1, 1], padding='SAME')+bias([32]))
# Pooling stage for first convolution which finds the maximum value from 2*2 kernel
    pool_conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
#2nd convolution layer    
    conv2 = tf.nn.relu(tf.nn.conv2d(pool_conv1,weight([5,5,32,64]), strides=[1, 1, 1, 1], padding='SAME')+bias([64]))
    pool_conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
#reshaping it back to 2d image and passing through a fully connected layer    
    pool_conv2_flat = tf.reshape(pool_conv2, [-1, 7*7*64])
    fc1 = tf.nn.relu(tf.matmul(pool_conv2_flat, weight([7*7*64,1024])) + bias([1024]))
#a dropout is applied after this to prevent overfitting of parameters due to strict training    
    keep_prob = tf.placeholder(tf.float32)   #it defines the probability of keeping certain features
    dropout = tf.nn.dropout(fc1, keep_prob)
#the final output layer with 10 classes denoting 0-9    
    y_conv = tf.matmul(dropout,weight([1024,10])) + bias([10])
#the probability and final output is returned
    return y_conv, keep_prob


In [6]:
#initializing variables x(input features), labels
x = tf.placeholder(tf.float32, [None, 784])
label = tf.placeholder(tf.float32, [None, 10])
y_conv, keep_prob = cnn(x)


In [7]:
cost = tf.nn.softmax_cross_entropy_with_logits(labels=label, logits=y_conv)
cost = tf.reduce_mean(cost)

optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(label, 1))
prediction = tf.cast(prediction, tf.float32)
accuracy = tf.reduce_mean(prediction)

In [9]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

for i in range(10000):
    batch_x, batch_y = mnist.train.next_batch(50)
    sess.run(optimizer, feed_dict={x: batch_x, label: batch_y, keep_prob: 0.5})

print('test accuracy %g' % accuracy.eval(feed_dict={
    x: mnist.test.images, label: mnist.test.labels, keep_prob: 1.0}))



test accuracy 0.9916
